In [479]:
import sys
import math
import cv2 as cv
import numpy as np
import pytesseract
from pytesseract import Output
import os
from icecream import ic
from PIL import Image
import pandas as pd
import subprocess
from datetime import date
from datetime import datetime

Reference  : https://www.youtube.com/watch?v=oXlwWbU8l2o&t=1071s

Load PYTesseract

In [480]:
os.environ["PATH"] = r"C:\Users\nandan.chitale\Documents\Projects\R&D\PDF Data reader python\Poppler\Release-23.07.0-0\poppler-23.07.0\Library\bin"
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

Read Image

In [481]:
img_path = os.path.join("pdf2Images")
output_path = os.path.join(img_path, "tmp", f"{datetime.today().strftime('%d_%b_%Y_%H_%M')}")
image_slice_path = os.path.join(output_path, "ocr_slices")

if not os.path.exists(output_path):
    os.mkdir(output_path)
    
if not os.path.exists(image_slice_path):
    os.mkdir(image_slice_path)
    
img_name = "page_1.jpg"

Save Original Image

In [482]:
def saveOriginalImg(img, index, write=True):
    cv.imwrite(os.path.join(output_path, f"{index}_original.png"), img)

GraySacle Image

In [483]:
def grayScaleImage(img, index, write=True):
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    if write:
        cv.imwrite(os.path.join(output_path, f"{index}_gray.png"), gray)
    return gray

Blur Grayscaled Image

In [484]:
def blurImage(img, index, write=True):
    blur = cv.GaussianBlur(img, (5,5), 0)
    if write:
        cv.imwrite(os.path.join(output_path, f"{index}_blur.png"), blur)
    return blur

Invert Grayscaled Image to read Lines accurately

In [485]:
def thrasholdImage(img, index, write=True):
    thresh = cv.threshold(img, 0,255, cv.THRESH_BINARY_INV + cv.THRESH_OTSU)[1]
    # thresh = cv.threshold(img, 127,255, cv.THRESH_BINARY)[1]
    if write:
        cv.imwrite(os.path.join(output_path, f"{index}_thresh.png"), thresh)
    return thresh

Invert Image

In [486]:
def invertImage(img, index, write=True):
    invert = cv.bitwise_not(img)
    if write:
        cv.imwrite(os.path.join(output_path, f"{index}_inverted.png"), invert)
    return invert

Erode Vertical Lines

In [487]:
def erodeVerticalLines(img, index, write=True):
    hor = np.array([[1,1,1,1,1,1]])
    vertial_lines_eroded_image = cv.erode(img, hor, iterations=10)
    vertial_lines_eroded_image = cv.dilate(vertial_lines_eroded_image, hor, iterations=10)
    
    if write:
        cv.imwrite(os.path.join(output_path, f"{index}_verode.png"), vertial_lines_eroded_image)
    return vertial_lines_eroded_image

Erode Horizontal Lines

In [488]:
def erodeHorizontalLines(img, index, write=True):
    ver = np.array([[1],[1],[1],[1],[1],[1]])
    horizontal_lines_eroded_image = cv.erode(img, ver, iterations=10)
    horizontal_lines_eroded_image = cv.dilate(horizontal_lines_eroded_image, ver, iterations=10)
    
    if write:
        cv.imwrite(os.path.join(output_path, f"{index}_herode.png"), horizontal_lines_eroded_image)
    return horizontal_lines_eroded_image

Combine Vertical and Horizontal Eroded Images

In [489]:
def combineErodedImages(vErodeImg, hErodeImg, index, write=True):
    combinedImages = cv.add(vErodeImg, hErodeImg)
    if write:
        cv.imwrite(os.path.join(output_path, f"{index}_eroded_combined.png"), combinedImages)
    return combinedImages

Dialate Combined Images to make lines thicker

In [490]:
def dialateImageToMakeLinesThicker(img, index, write=True):
    kernal = cv.getStructuringElement(cv.MORPH_RECT, (2,2))
    combineImageDialated = cv.dilate(img, kernal, iterations=5)
    if write:
        cv.imwrite(os.path.join(output_path, f"{index}_combinedDialated.png"), combineImageDialated)
    return combineImageDialated

In [491]:
def combineRemovedNoiseLineImage(imgWithoutLineNoise, imgWithLinesEnhanced, index, write=True):
    combinedImages = cv.add(imgWithoutLineNoise, imgWithLinesEnhanced)
    if write:
        cv.imwrite(os.path.join(output_path, f"{index}_combineRemovedNoiseLineImage.png"), combinedImages)
    return combinedImages

Subtract combined and dilated image from original image

In [492]:
def subtractCombinedDilatedFromOriginalImage(invertedImg, dilatedCombinedImg, index, write=True):
    imgWithoutLines = cv.subtract(invertedImg, dilatedCombinedImg)
    if write:
        cv.imwrite(os.path.join(output_path, f"{index}_imgWithoutLines.png"), imgWithoutLines)
    return imgWithoutLines

Remove noise with erode and dilate

In [493]:
def removeNoiseWithErodeAndDilate(img, index, write=True):
    kernal = cv.getStructuringElement(cv.MORPH_RECT, (2,2))
    imgWithoutLinesNoiseRemoved = cv.erode(img, kernal, iterations=1)
    imgWithoutLinesNoiseRemoved = cv.dilate(imgWithoutLinesNoiseRemoved, kernal, iterations=1)
    if write:
        cv.imwrite(os.path.join(output_path, f"{index}_removeNoiseWithErodeAndDilate.png"), imgWithoutLinesNoiseRemoved)
    return imgWithoutLinesNoiseRemoved

Dilate Image
(Pixels are added when structuring elements hits part of image)

In [494]:
def dilateImage(thrasholdImg, index, write=True):
    kernel_to_remove_gaps_between_words = np.array([
            # [1,1,1,1,1,1,1,1,1,1],
            # [1,1,1,1,1,1,1,1,1,1]
            [1,1],
            [1,1]
    ])
    
    dilated_image_1 = cv.dilate(thrasholdImg, kernel_to_remove_gaps_between_words, iterations=5)
    simple_kernel = np.ones((5,5), np.uint8)
    dilated_image = cv.dilate(dilated_image_1, simple_kernel, iterations=2)
    if write:
        cv.imwrite(os.path.join(output_path, f"{index}_dilateImage1.png"), dilated_image_1)
        cv.imwrite(os.path.join(output_path, f"{index}_dilateImage.png"), dilated_image)
    return dilated_image

Find BLOB Contours

In [495]:
def findContours(img, originalImg, index, write=True):
    result = cv.findContours(img, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)
    contours = result[0]
    
    imgWithControusDrawn = originalImg.copy()
    cv.drawContours(imgWithControusDrawn, contours, -1, (0, 255, 0), 2)
    if write:
        cv.imwrite(os.path.join(output_path, f"{index}_imgWithControusDrawn.png"), imgWithControusDrawn)
    return contours

Convert contours to bounding boxes

In [496]:
def convert_contours_to_bounding_boxes(contours, original_Img, index, write=True):
    bounding_boxes = []
    image_with_all_bounding_boxes = original_Img.copy()
    for contour in contours:
        x, y, w, h = cv.boundingRect(contour)
        bounding_boxes.append((x, y, w, h))
        # This line below is about
        # drawing a rectangle on the image with the shape of
        # the bounding box. Its not needed for the OCR.
        # Its just added for debugging purposes.
        image_with_all_bounding_boxes = cv.rectangle(image_with_all_bounding_boxes, (x, y), (x + w, y + h), (0, 255, 0), 3)
        
        
    if write:
            cv.imwrite(os.path.join(output_path, f"{index}_image_with_all_bounding_boxes.png"), image_with_all_bounding_boxes)
            
    return bounding_boxes

#### Sorting The Bounding Boxes By X And Y Coordinates To Make Rows And Columns

get mean height of bounding box

In [497]:
def get_mean_height_of_bounding_boxes(bounding_boxes):
    heights = []
    for bounding_box in bounding_boxes:
        x, y, w, h = bounding_box
        heights.append(h)
    return np.mean(heights)

sort_bounding_boxes_by_y_coordinate

In [498]:
def sort_bounding_boxes_by_y_coordinate(bounding_boxes):
    bounding_boxes = sorted(bounding_boxes, key=lambda x: x[1])
    return bounding_boxes

Club bounding boxes by similar y co-ordinates into rows

In [499]:
def club_all_bounding_boxes_by_similar_y_coordinates_into_rows(mean_height, bounding_boxes):
    rows = []
    half_of_mean_height = mean_height / 2
    current_row = [ bounding_boxes[0] ]
    for bounding_box in bounding_boxes[1:]:
        current_bounding_box_y = bounding_box[1]
        previous_bounding_box_y = current_row[-1][1]
        distance_between_bounding_boxes = abs(current_bounding_box_y - previous_bounding_box_y)
        if distance_between_bounding_boxes <= half_of_mean_height:
            current_row.append(bounding_box)
        else:
            rows.append(current_row)
            current_row = [ bounding_box ]
    rows.append(current_row)
    
    return rows

Sort all rows by x co-ordinates

In [500]:
def sort_all_rows_by_x_coordinate(rows):
    for row in rows:
        row.sort(key=lambda x: x[0])
    return rows

Teserract Config

In [501]:
tesseractConfig = r"--dpi 72 --psm 10 --oem 3 tessedit_char_whitelist=ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789"

Serach Result from py teserractx

In [502]:
def get_result_from_tersseract(image_path):
    # output = subprocess.getoutput('tesseract ' + image_path + ' - -l eng --oem 3 --psm 7 --dpi 72 -c tessedit_char_whitelist="ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789().calmg* "')
    
    # Convert text to boxes using teserract
    # output = pytesseract.image_to_boxes(image_path, lang='eng', config="--psm 7 --dpi 72")
    
    # Draw boxes around text generated by teserract
    # img = cv.imread(os.path.join(image_path))
    # height, width, _ = img.shape
    # teserractImg = img.copy()
        
    output = pytesseract.image_to_string(image_path, config=tesseractConfig, output_type=Output.STRING)
    # amount_boxes = len(data['text'])
    # for i in range(amount_boxes):
    #     if(float(data['conf'][i])) > 70:
    #         (x,y,width,height) = (data['left'][i],data['top'][i],data['width'][i],data['height'][i])
    #         cv.rectangle(teserractImg, (x,y), (x+width, y+height), (0,255,0), 2)
                
    # cv.imwrite(os.path.join(output_path, f"teserractImg.png"), teserractImg)
    output = output.strip()
    return output

Extracting text from box using OCR

In [503]:
def crop_each_bounding_box_and_ocr(rows, original_image):
    table = []
    current_row = []
    image_number = 0
    for row in rows:
        for bounding_box in row:
            x, y, w, h = bounding_box
            y = y - 5
            cropped_image = original_image[y:y+h, x:x+w]
            sliced_img_path = os.path.join(image_slice_path, f"{str(image_number)}.jpg")
            cv.imwrite(sliced_img_path, cropped_image)
            results_from_ocr = get_result_from_tersseract(sliced_img_path)
            current_row.append(results_from_ocr)
            image_number += 1
        table.append(current_row)
        current_row = []
        
    return table

Generating csv file of output

In [504]:
def generate_excel_file(table):
    df = pd.DataFrame(table)
    with pd.ExcelWriter(os.path.join(output_path,"output.xlsx")) as writer:
        df.to_excel(writer, sheet_name='sheet1', index=False)

In [505]:
originalImg = cv.imread(os.path.join(img_path, img_name))

In [506]:
# Save Original Image
saveOriginalImg(originalImg, 1)

# Grayscale image
img = grayScaleImage(originalImg, index=2)

# Thrashold Image
thrasholdImage = thrasholdImage(img, index=3)

# Invert image
# invertedImg = invertImage(thrasholdImage, index=4)

# Erode vertical lines
vErodeImg = erodeVerticalLines(thrasholdImage, index=5)

# Erode Horizontal lines
hErodeImg = erodeHorizontalLines(thrasholdImage, index=6)

# Combine Eroded Images
img = combineErodedImages(vErodeImg, hErodeImg, index=7)

# Dilate Combined Image to make lines thicker
linesDialatedImage = dialateImageToMakeLinesThicker(img, index=8)

# Remove the lines
img = subtractCombinedDilatedFromOriginalImage(thrasholdImage , linesDialatedImage, index=9)

# Remove Noise with Erode and Dilate
noiseRemovedImg = removeNoiseWithErodeAndDilate(img, index=10)

# Combine Removed noise image with lines image
# img = combineRemovedNoiseLineImage(noiseRemovedImg, linesDialatedImage, index=11)

# Dilate Image
img = dilateImage(noiseRemovedImg, index=12)

# Contours
contours = findContours(img, originalImg , index=13)

# Convert contours to boxes
boundingBoxes = convert_contours_to_bounding_boxes(contours, originalImg, index=14)


In [507]:
# Get mean height of boxes
meanHeight = get_mean_height_of_bounding_boxes(boundingBoxes)

# Sort bounding boxes
boundingBoxes = sort_bounding_boxes_by_y_coordinate(boundingBoxes)

# Club all bounding boxes by similar y co-ordinates
rows = club_all_bounding_boxes_by_similar_y_coordinates_into_rows(meanHeight, boundingBoxes)

# Sort rows by x co-ordinate
rows = sort_all_rows_by_x_coordinate(rows)

# Crop each box and extract text using ocr
table = crop_each_bounding_box_and_ocr(rows, originalImg)

ic(table)

ic| table: [['=', '>', 'J', 'R', 'C', '-|', 'Ly', 'S', '—', ')', 'R', 'D', '=', '4'],
            ['Seer',
             'aaron',
             'TO',
             'g',
             'IPCA',
             'LABORATORIES',
             'ORDER',
             'NO.',
             'i',
             '63'],
            ['4', 'To 6,', ','],
            ['Dhanlaxmi', 'Plaza,', '796,', 'ORDER DT.', 'e', '12/04/2023'],
            ['Sadashiv Peth', 'Pune Pincode - 411030 (Maharashtra)', 'ee ee.'],
            ['Tele',
             '.',
             '020-24482021 22,23',
             '_',
             'i',
             'Mobile',
             '.',
             '8308583200-DESAI',
             'D',
             'SUPPLY DT.:',
             '12/04/2023'],
            ['Email', ':', '[price = ctgaal'],
            ['Email',
             ': shriramenterprises1992@gmail.com',
             'MFGR',
             '2',
             'es',
             'L DA',
             '“an',
             'TRPT NAME',
           

[['=', '>', 'J', 'R', 'C', '-|', 'Ly', 'S', '—', ')', 'R', 'D', '=', '4'],
 ['Seer',
  'aaron',
  'TO',
  'g',
  'IPCA',
  'LABORATORIES',
  'ORDER',
  'NO.',
  'i',
  '63'],
 ['4', 'To 6,', ','],
 ['Dhanlaxmi', 'Plaza,', '796,', 'ORDER DT.', 'e', '12/04/2023'],
 ['Sadashiv Peth', 'Pune Pincode - 411030 (Maharashtra)', 'ee ee.'],
 ['Tele',
  '.',
  '020-24482021 22,23',
  '_',
  'i',
  'Mobile',
  '.',
  '8308583200-DESAI',
  'D',
  'SUPPLY DT.:',
  '12/04/2023'],
 ['Email', ':', '[price = ctgaal'],
 ['Email',
  ': shriramenterprises1992@gmail.com',
  'MFGR',
  '2',
  'es',
  'L DA',
  '“an',
  'TRPT NAME',
  'ie',
  'OM',
  'SAI'],
 ['DLNo.',
  '5',
  '20B-PZ1-200017 - 21B-PZ1-200019 - 20D-PZ1-200018',
  'DLNo.',
  'a',
  '20B-MH-PZ3-178658 B21-MH-PZ3-178659',
  'TRANSPORT'],
 ['GSTIN : 27AAEFS1105J1ZF', 'GSTIN : 27AAACI1220M1ZT'],
 ['FSSAI No.:', '11512020006253', 'LBTNo. :'],
 ['Dear Sir/Madam,'],
 ['Kindy Arrange To Supply As Per Below Mentioned Order By Our Registered Transport/Co

In [508]:
# Generate csv file
generate_excel_file(table)